In [1]:
from dlcliche.notebook import *
from dlcliche.utils import *

In [2]:
import torchvision
import logging
from sklearn.metrics import average_precision_score


logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [3]:
EVEN, ODD = 10, 11


class MultiLabelMNIST(torchvision.datasets.MNIST):
    def __init__(self, folder, train, download=False):
        super().__init__(folder, train=train, download=download)

    def __getitem__(self, index):
        x, y = super().__getitem__(index)
        label = [0,0,0,0,0, 0,0,0,0,0, 0,0]
        # multi label
        label[y] = 1 # one of class 0-9
        label[ODD if y % 2 == 1 else EVEN] = 1 # odd or even
        return x, label


org_train = MultiLabelMNIST('data', train=True, download=True)
org_test = MultiLabelMNIST('data', train=False)
X = org_train.data.view(-1, 28*28).numpy()
y = [org_train[i][1] for i in range(len(org_train))]
test_X = org_test.data.view(-1, 28*28).numpy()
test_y = [org_test[i][1] for i in range(len(org_test))]

## TorchMLPClassifier

In [4]:
from MLP.torch_mlp_clf import TorchMLPClassifier

clf = TorchMLPClassifier(debug=True)
clf.fit(X, y)
print(clf.score(test_X, test_y))

# mAP
preds = clf.predict(test_X, multi_label_n_class=12)
print('mAP:', average_precision_score(test_y, preds))

for i in range(preds.shape[1]):
    print(i, 'precision:', average_precision_score(np.array(test_y)[:, i], preds[:, i]))

MLP(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=12, bias=True)
  )
)
mAP BCEWithLogitsLoss() 12 <class 'torch.Tensor'>


INFO:MLP.torch_mlp_clf:epoch 0001/200: lr: 0.0010000: loss=24.287267 val_mAP=0.9754035 val_loss=10.9644251
INFO:MLP.torch_mlp_clf:epoch 0002/200: lr: 0.0010000: loss=9.447199 val_mAP=0.9843268 val_loss=8.5380964
INFO:MLP.torch_mlp_clf:epoch 0003/200: lr: 0.0010000: loss=7.213789 val_mAP=0.9879271 val_loss=7.3381443
INFO:MLP.torch_mlp_clf:epoch 0004/200: lr: 0.0010000: loss=5.886614 val_mAP=0.9900838 val_loss=6.4805279
INFO:MLP.torch_mlp_clf:epoch 0005/200: lr: 0.0010000: loss=5.003161 val_mAP=0.9902492 val_loss=6.3941507
INFO:MLP.torch_mlp_clf:epoch 0006/200: lr: 0.0010000: loss=4.348244 val_mAP=0.9916821 val_loss=6.0478668
INFO:MLP.torch_mlp_clf:epoch 0007/200: lr: 0.0010000: loss=3.773977 val_mAP=0.9920540 val_loss=5.9201288
INFO:MLP.torch_mlp_clf:epoch 0008/200: lr: 0.0010000: loss=3.327906 val_mAP=0.9917956 val_loss=5.9784708
INFO:MLP.torch_mlp_clf:epoch 0009/200: lr: 0.0010000: loss=2.977687 val_mAP=0.9912087 val_loss=6.1600981
INFO:MLP.torch_mlp_clf:epoch 0010/200: lr: 0.0010000:

0.9918656498154285
mAP: 0.9918656498154285
0 precision: 0.9975848850647593
1 precision: 0.9977354667745668
2 precision: 0.9905144917078703
3 precision: 0.9917295357466553
4 precision: 0.9918822288272158
5 precision: 0.9885295689128226
6 precision: 0.9952459059907298
7 precision: 0.9831979480659762
8 precision: 0.9857008053708226
9 precision: 0.9871498797419745
10 precision: 0.9973152187629448
11 precision: 0.9958018628188042


## Normal MLPClassifier

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
test_X = scaler.transform(test_X)

clf = MLPClassifier()
clf.fit(X, y)
clf.score(test_X, test_y)

0.9449

In [5]:
# mAP
preds = clf.predict(test_X)
print('mAP:', average_precision_score(test_y, preds))

for i in range(preds.shape[1]):
    print(i, 'precision:', average_precision_score(np.array(test_y)[:, i], preds[:, i]))

mAP: 0.9474026367800934
0 precision: 0.9628405304861388
1 precision: 0.9777427576937121
2 precision: 0.9368970334509699
3 precision: 0.9338057461457567
4 precision: 0.93732925778772
5 precision: 0.9328777249016199
6 precision: 0.9509429714683367
7 precision: 0.9358512465490314
8 precision: 0.9264177220742398
9 precision: 0.9219176843280218
10 precision: 0.9751203970268995
11 precision: 0.9770885694486754
